# HTML Generator

In this example, we'll build an app that automatically generates HTML components, evaluates them, and captures user feedback. We'll use the feedback and evaluations to build up a dataset
that we'll use as a basis for further improvements.


## The generator

We'll start by using a very simple prompt to generate HTML components using `gpt-3.5-turbo`.


In [1]:
// Initialize an openai client and wrap it with Braintrust's helper. This is a no-op until we start using
// the client within code that is instrumented by Braintrust.
import { OpenAI } from "openai";
import { wrapOpenAI } from "braintrust";

const openai = wrapOpenAI(
  new OpenAI({
    apiKey: process.env.OPENAI_API_KEY || "Your OPENAI_API_KEY",
  })
);


In [6]:
import { ChatCompletionMessageParam } from "openai/resources";

function generateMessages(input: string): ChatCompletionMessageParam[] {
  return [
    {
      role: "system",
      content: `You are a skilled design engineer
who can convert ambiguously worded ideas into beautiful, crisp HTML and CSS.
Your designs value simplicity, conciseness, clarity, and functionality over
complexity.

You generate pure HTML with inline CSS, so that your designs can be rendered
directly as plain HTML.

Users will send you a description of a design, and you must reply with HTML,
and nothing else. Your reply will be directly copied and rendered into a browser,
so do not include any text. If you would like to explain your reasoning, feel free
to do so in HTML comments.`,
    },
    {
      role: "user",
      content: input,
    },
  ];
}

JSON.stringify(
  generateMessages("A login form for a B2B SaaS product."),
  null,
  2
);


[
  {
    "role": "system",
    "content": "You are a skilled design engineer\nwho can convert ambiguously worded ideas into beautiful, crisp HTML and CSS.\nYour designs value simplicity, conciseness, clarity, and functionality over\ncomplexity.\n\nYou generate pure HTML with inline CSS, so that your designs can be rendered\ndirectly as plain HTML.\n\nUsers will send you a description of a design, and you must reply with HTML,\nand nothing else. Your reply will be directly copied and rendered into a browser,\nso do not include any text. If you would like to explain your reasoning, feel free\nto do so in HTML comments."
  },
  {
    "role": "user",
    "content": "A login form for a B2B SaaS product."
  }
]


Now, let's run this using `gpt-3.5-turbo`. We'll also do a few things that help us log & evaluate this function later:

- Wrap the execution in a `traced` call, which will enable Braintrust to log the inputs and outputs of the function when we run it in production or in evals
- Make its signature accept a single `input` value, which Braintrust's `Eval` function expects


In [7]:
import { traced } from "braintrust";
async function generateComponent(input: string) {
  return traced(
    async (span) => {
      const response = await openai.chat.completions.create({
        model: "gpt-3.5-turbo",
        messages: generateMessages(input),
        seed: 101,
      });
      const output = response.choices[0].message.content;
      span.log({ input, output });
      return output;
    },
    {
      name: "generateComponent",
    }
  );
}

(await generateComponent("A login form for a B2B SaaS product."));


<form>
  <label for="username">Username</label>
  <input type="text" id="username" name="username">

  <label for="password">Password</label>
  <input type="password" id="password" name="password">

  <input type="submit" value="Login">
</form>


Let's look at a few examples!


In [8]:
import * as tslab from "tslab";

async function displayComponent(input: string) {
    const result = await generateComponent(input);
    tslab.display.html(result);
    console.log("\n");
    console.log(result);
}

await displayComponent("A login form for a B2B SaaS product.");

Username 
 

 Password



<form>
  <label for="username">Username</label>
  <input type="text" id="username" name="username">

  <label for="password">Password</label>
  <input type="password" id="password" name="password">

  <input type="submit" value="Login">
</form>


In [9]:
await displayComponent("A pricing page for a rideshare app. Make heavy use of dark mode.");

<!DOCTYPE html>
 
 
 
 
 Rideshare App - Pricing 
 
 
 
 
 Pricing 
 
 Basic 
 $10/month 
 2 rides per month 
 Get started 
 
 
 Pro 
 $20/month 
 Unlimited rides 
 Get started 
 
 
 Premium 
 $30/month 
 Unlimited rides, priority support 
 Get started 
 
 © 2022 Rideshare App. All rights reserved.



<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Rideshare App - Pricing</title>
  <style>
    body {
      background-color: black;
      color: white;
      font-family: Arial, sans-serif;
    }

    .container {
      display: flex;
      flex-direction: column;
      align-items: center;
      justify-content: center;
      height: 100vh;
    }

    h1 {
      font-size: 2rem;
      margin-bottom: 1.5rem;
    }

    .pricing-card {
      background-color: #262626;
      padding: 2rem;
      width: 400px;
      text-align: center;
      margin-bottom: 1rem;
    }

    .pricing-card h3 {
      margin-top: 0;
      font-size: 1.5rem;
      margin-bottom: 1rem;
    }

    .pricing-card p {
      margin: 0;
      font-size: 1.2rem;
      opacity: 0.7;
    }

    .pricing-card .price {
      font-size: 2rem;
      margin-bottom: 1rem;
    }

    .pricing-card .btn {
      background-color:

In [10]:
await displayComponent("Logs viewer for a cloud infrastructure management tool.");

<!DOCTYPE html>
 
 
 
 Logs Viewer 
 
 
 
 Logs Viewer 
 
 
 2022-10-01 10:30:00 
 Info: System initialized 
 
 
 2022-10-01 10:31:05 
 Warning: High CPU usage detected 
 
 
 2022-10-01 10:32:20 
 Error: Failed to connect to database 
 
 <!-- Add more logs here -->



<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Logs Viewer</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 20px;
        }

        h1 {
            color: #333;
            font-size: 24px;
            margin-bottom: 20px;
        }

        .logs-container {
            border: 1px solid #ccc;
            padding: 10px;
        }

        .log {
            margin-bottom: 10px;
            padding: 10px;
            background-color: #f7f7f7;
            border: 1px solid #ddd;
            border-radius: 4px;
        }

        .timestamp {
            color: #999;
            font-size: 14px;
        }

        .message {
            color: #333;
            font-size: 16px;
            margin-top: 5px;
        }
    </style>
</head>
<body>
    <h1>Logs Viewer</h1>
    <div class="logs-container">
        <div class="log">
            <div class="timestamp">2022-10-01 10:30:00</div>
   

It looks like in a few of these examples, the model is generating dark text on a dark background, e.g. "Logs Viewer". This seems like a case we could evaluate and make sure does not happen.

To do that, we'll use [puppeteer](https://pptr.dev/) to render the HTML and then extract some styling data we can evaluate. The puppeteer code is implemented in a separate file ([render.js](/render.js)) which is included alongside this example.

In [32]:
import { getComputedTextStyles } from "./render";
await getComputedStyles(await generateComponent("Logs viewer for a cloud infrastructure management tool."));


[
  {
    element: 'H1',
    color: 'rgb(51, 51, 51)',
    backgroundColor: 'rgba(0, 0, 0, 0)',
    text: 'Logs Viewer'
  },
  {
    element: 'DIV',
    color: 'rgb(153, 153, 153)',
    backgroundColor: 'rgba(0, 0, 0, 0)',
    text: '2022-10-01 10:30:00'
  },
  {
    element: 'DIV',
    color: 'rgb(51, 51, 51)',
    backgroundColor: 'rgba(0, 0, 0, 0)',
    text: 'Info: System initialized'
  },
  {
    element: 'DIV',
    color: 'rgb(153, 153, 153)',
    backgroundColor: 'rgba(0, 0, 0, 0)',
    text: '2022-10-01 10:31:05'
  },
  {
    element: 'DIV',
    color: 'rgb(51, 51, 51)',
    backgroundColor: 'rgba(0, 0, 0, 0)',
    text: 'Warning: High CPU usage detected'
  },
  {
    element: 'DIV',
    color: 'rgb(153, 153, 153)',
    backgroundColor: 'rgba(0, 0, 0, 0)',
    text: '2022-10-01 10:32:20'
  },
  {
    element: 'DIV',
    color: 'rgb(51, 51, 51)',
    backgroundColor: 'rgba(0, 0, 0, 0)',
    text: 'Error: Failed to connect to database'
  }
]


In [35]:
function getLuminance(color) {
    var rgba = color.match(/\d+/g);
    for (var i = 0; i < 3; i++) {
        var rgb = rgba[i];
        rgb /= 255;
        rgb = rgb < 0.03928 ? rgb / 12.92 : Math.pow((rgb + 0.055) / 1.055, 2.4);
        rgba[i] = rgb;
    }
    return 0.2126 * rgba[0] + 0.7152 * rgba[1] + 0.0722 * rgba[2];
}

function getContrastRatio(luminance1, luminance2) {
    var brighter = Math.max(luminance1, luminance2);
    var darker = Math.min(luminance1, luminance2);
    return (brighter + 0.05) / (darker + 0.05);
}

getContrastRatio(getLuminance('rgb(51, 51, 51)'), getLuminance('rgba(0, 0, 0, 0)'))

1.6620953314177012
